In [28]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as A
import pandas as pd
from sklearn.manifold import TSNE
import sklearn.manifold 
from matplotlib import pyplot as plt
import re
from numpy  import array
from scipy import stats
from scipy.spatial import distance
import os
import codecs
from sklearn.metrics.pairwise import cosine_similarity

# TensoBoardX

In [21]:
corpus_filename = '../data/extracted_emoji_sequences.txt' 

In [22]:
corpus = open(corpus_filename).read().splitlines()

In [23]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

# this is just the very basic translation both ways plus the length of vocabulary


In [26]:
loadedLayer = torch.load(os.path.join("SOFTMAX_trained", "1epoch#2.pt"))

In [71]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:        
        cosineSimilarity = cosine_similarity(loadedLayer.detach().numpy()[word2idx[emoji1]].reshape(-1,300), loadedLayer.detach().numpy()[word2idx[emoji2]].reshape(-1,300))[0][0]
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient ist {}'.format(spearmanRank))

the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
Der Spearman Rank Correlation Coefficient ist SpearmanrResult(correlation=0.19603981256012548, pvalue=1.0285218117520258e-05)


/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [50]:
cosine_similarity(loadedLayer.detach().numpy()[word2idx['🎊']].reshape(-1,300), loadedLayer.detach().numpy()[word2idx['🎉']].reshape(-1,300))[0][0]

-0.056822285

In [52]:
loadedLayer.detach().numpy()[word2idx['🎊']].reshape(-1,300)

array([[ 2.79803365e-01,  1.08051419e+00,  4.10220593e-01,
         2.63290226e-01, -1.15821235e-01,  8.47545207e-01,
        -1.52178437e-01, -3.57537687e-01, -5.34075439e-01,
        -1.23984389e-01, -1.24372333e-01, -4.73752350e-01,
         6.53865188e-02, -1.88909564e-02, -2.91305244e-01,
        -2.21792400e-01,  3.31730932e-01, -2.45904788e-01,
         8.14972162e-01, -1.09888995e-02, -7.69089162e-01,
        -3.01964581e-01, -1.84197128e+00, -5.72287083e-01,
        -9.40967202e-01,  1.66754711e+00,  2.17138276e-01,
         9.64368463e-01,  4.54514891e-01,  8.76853228e-01,
        -7.52373993e-01, -4.13238741e-02,  4.82867956e-01,
         1.62349626e-01, -1.20415592e+00, -1.41218674e+00,
        -1.90491164e+00, -3.98953825e-01, -1.12450123e+00,
        -1.40908289e+00,  4.49220911e-02,  1.23544598e+00,
         1.01461542e+00,  1.75786233e+00, -1.43848038e+00,
         3.49596083e-01,  3.64895836e-02, -1.33346057e+00,
         1.39005929e-01,  8.45441967e-02,  5.65456629e-0

In [16]:
#     from tensorboardX import SummaryWriter
#SummaryWriter encapsulates everything
# writer = SummaryWriter('runs/exp-2')
#creates writer object. The log will be saved in 'runs/exp-1'
#     testWriter = SummaryWriter()
#creates writer2 object with auto generated file name, the dir will be something like 'runs/Aug20-17-20-33'
# writer3 = SummaryWriter(comment='3x learning rate')
#creates writer3 object with auto generated file name, the comment will be appended to the filename. The dir will be something like 'runs/Aug20-17-20-33-3xlearning rate'

In [17]:
#     testWriter.close

In [33]:
# from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([[3, 4, 5]], [[6,78, 898]])

array([[0.75621265]])